In [123]:
import numpy as np
import pandas as pd 
import math
import random

In [124]:
# 朴素贝叶斯分类器实现
# 导入训练文本分类数据
def loadDataSet():
    # 词条切分后的文档集合，列表每一行代表一个email
    postingList = [
        ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
        ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
        ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
        ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
        ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
        ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
    ]
    # 由人工标注的每篇文档的类标签
    classVec = [0, 1, 0, 1, 0, 1]  # 1代表侮辱性文字，0代表正常言论
    return postingList, classVec


postingList, classVec = loadDataSet()


# 统计所有文档中出现的词条列表
def createVocabList(dataSet):
    vocabSet = set([])
    # 遍历文档集合中的每一篇文档
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)


vocabSet = createVocabList(postingList)
vocabSet


# 根据词条列表中的词条是否在文档中出现(出现1，未出现0)，将文档转化为词条向量
def setOfWords2Vec(vocabSet, inputSet):
    # 新建一个长度为vocabSet的列表，并且各维度元素初始化为0
    returnVec = [0] * len(vocabSet)
    # 遍历文档中的每一个词条
    for word in inputSet:
        # 如果词条在词条列表中出现
        if word in vocabSet:
            # 通过列表获取当前word的索引(下标)
            # 将词条向量中的对应下标的项由0改为1
            returnVec[vocabSet.index(word)] = 1
        else:
            print('the word: %s is not in my vocabulary! ' % 'word')
    # 返回inputet转化后的词条向量
    return returnVec


trainMatrix = [setOfWords2Vec(vocabSet, inputSet) for inputSet in postingList]

# trainMatrix


# 训练算法，从词向量计算概率p(w0|ci)...及p(ci)
# @trainMatrix：由每篇文档的词条向量组成的文档矩阵
# @trainCategory: 每篇文档的类标签组成的向量
def trainNB0(trainMatrix, trainCategory):
    # 获取文档矩阵中文档的数目
    numTrainDocs = len(trainMatrix)
    # 获取词条向量的长度
    numWords = len(trainMatrix[0])
    # 所有文档中属于类1所占的比例p(c=1)
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    # 创建一个长度为词条向量等长的列表
    p0Num = np.ones(numWords)  # ham 1x32
    p1Num = np.ones(numWords)  # spam 1x32
    p0Denom = 2.0
    p1Denom = 2.0
    #遍历每一篇文档的词条向量
    for i in range(numTrainDocs):
        # 如果该词条向量对应的标签为1
        if trainCategory[i] == 1:
            # 统计所有类别为1的词条向量中各个词条出现的次数
            p1Num += trainMatrix[i]
            # 统计类别为1的词条向量中出现的所有词条的总数
            # 即统计类1所有文档中出现单词的数目
            p1Denom += sum(trainMatrix[i])
        else:
            # 统计所有类别为0的词条向量中各个词条出现的次数
            p0Num += trainMatrix[i]
            # 统计类别为0的词条向量中出现的所有词条的总数
            # 即统计类0所有文档中出现单词的数目
            p0Denom += sum(trainMatrix[i])


#     return p1Num, p1Denom, p0Num, p0Denom
# 利用NumPy数组计算p(wi|c1)
    p1Vect = np.log(p1Num / p1Denom)  # 为避免下溢出问题，需要改为log()
    # 利用NumPy数组计算p(wi|c0)
    p0Vect = np.log(p0Num / p0Denom)  # 为避免下溢出问题，需要改为log()
    return p0Vect, p1Vect, pAbusive

p0Vect, p1Vect, pAbusive = trainNB0(trainMatrix, classVec)
trainNB0(trainMatrix, classVec)


# 朴素贝叶斯分类函数
# @vec2Classify:待测试分类的词条向量
# @p0Vec:类别0所有文档中各个词条出现的频数p(wi|c0)
# @p1Vec:类别1所有文档中各个词条出现的频数p(wi|c1)
# @pClass1:类别为1的文档占文档总数比例
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    # 根据朴素贝叶斯分类函数分别计算待分类文档属于类1和类0的概率
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 'spam'
    else:
        return 'not spam'


# setOfWords2Vec改造
def bagOfWords2VecMN(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
    return returnVec


# 测试朴素贝叶斯分类函数
def testingNB():
    listOPosts, listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V, p1V, pAb = trainNB0(np.array(trainMat), np.array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry, 'classified as: ', classifyNB(thisDoc, p0V, p1V, pAb))


testingNB()

['so',
 'cute',
 'I',
 'quit',
 'has',
 'please',
 'stop',
 'how',
 'help',
 'take',
 'park',
 'stupid',
 'buying',
 'ate',
 'garbage',
 'him',
 'to',
 'worthless',
 'is',
 'licks',
 'steak',
 'posting',
 'dog',
 'not',
 'maybe',
 'dalmation',
 'mr',
 'flea',
 'love',
 'problems',
 'my',
 'food']

(array([-2.56494936, -2.56494936, -2.56494936, -3.25809654, -2.56494936,
        -2.56494936, -2.56494936, -2.56494936, -2.56494936, -3.25809654,
        -3.25809654, -3.25809654, -3.25809654, -2.56494936, -3.25809654,
        -2.15948425, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
        -2.56494936, -3.25809654, -2.56494936, -3.25809654, -3.25809654,
        -2.56494936, -2.56494936, -2.56494936, -2.56494936, -2.56494936,
        -1.87180218, -3.25809654]),
 array([-3.04452244, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
        -3.04452244, -2.35137526, -3.04452244, -3.04452244, -2.35137526,
        -2.35137526, -1.65822808, -2.35137526, -3.04452244, -2.35137526,
        -2.35137526, -2.35137526, -1.94591015, -3.04452244, -3.04452244,
        -3.04452244, -2.35137526, -1.94591015, -2.35137526, -2.35137526,
        -3.04452244, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
        -3.04452244, -2.35137526]),
 0.5)

['love', 'my', 'dalmation'] classified as:  not spam
['stupid', 'garbage'] classified as:  spam


In [126]:
# 朴素贝叶斯分类器应用过滤垃圾邮件
def textParse(bigString):  # input is big string, # output is word list
    import re
    listOfTokens = re.split(r'\W*', bigString)  # 正则分割
    # 去除少于两个字符的字符串，大写转为小写
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]


def spamTest():
    docList = []
    classList = []
    fullText = []
    # 导入文本文件，解析为词条列表
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)  # 统计所有文档中出现的词条列表
    # 构建训练街、测试集
    trainingSet = list(range(50))
    testSet = []  # 创建测试集
    for i in list(range(10)):
        randIndex = int(random.uniform(0, len(trainingSet)))
        # uniform() 方法将随机生成下一个实数，它在 [x, y] 范围内
        testSet.append(trainingSet[randIndex])
        del (trainingSet[randIndex])
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        # 将文档转化为词条向量并添加至训练集
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    # 训练算法，从词向量计算概率p(w0|ci)...及p(ci)
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    # 测试集
    errorCount = 0
    for docIndex in testSet:
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(np.array(wordVector), p0V, p1V,
                      pSpam) != classList[docIndex]:
            errorCount += 1
            print("classification error", docList[docIndex])
    print('the error rate is: ', float(errorCount) / len(testSet))
    #return vocabList,fullText


spamTest()

classification error ['you', 'have', 'everything', 'gain', 'incredib1e', 'gains', 'length', 'inches', 'yourpenis', 'permanantly', 'amazing', 'increase', 'thickness', 'yourpenis', 'betterejacu1ation', 'control', 'experience', 'rock', 'harderecetions', 'explosive', 'intenseorgasns', 'increase', 'volume', 'ofejacu1ate', 'doctor', 'designed', 'and', 'endorsed', '100', 'herbal', '100', 'natural', '100', 'safe', 'the', 'proven', 'naturalpenisenhancement', 'that', 'works', '100', 'moneyback', 'guaranteeed']
classification error ['you', 'have', 'everything', 'gain', 'incredib1e', 'gains', 'length', 'inches', 'yourpenis', 'permanantly', 'amazing', 'increase', 'thickness', 'yourpenis', 'betterejacu1ation', 'control', 'experience', 'rock', 'harderecetions', 'explosive', 'intenseorgasns', 'increase', 'volume', 'ofejacu1ate', 'doctor', 'designed', 'and', 'endorsed', '100', 'herbal', '100', 'natural', '100', 'safe', 'the', 'proven', 'naturalpenisenhancement', 'that', 'works', '100', 'moneyback', 'gu

D:\Program Files\anaconda3\lib\re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)
